### kernel density estimation and kernel regression for prediction
This notebook is my first attempt to write down the ddiff and dddiff models that I have been working on

mixed recursive/iterative approach (rather than vectorized approach that is harder to generalize
- create separate submodules for KDE and KDreg
    - develop a system for storing differenced but unweighted data and the appropriate mask to lessen memory usage. 
        -1's or 0's depending on i/j/k/l/.... values
    - figure out the deepest level requested by the user and levels that are less deep are just slices of the deeper model. 
        -scout out the tree to build the differenced dataset. 
          
        
 plot and compare
on synthetic data, 1, 2, 3+ mixed distributions
1d - Ndiff vs gaussian kernel vs kernel_tunneling
2d -  


multidimensional x problem
e.g., parameter treatment
"product kernel approach" vs l2 "el two" (radial basis?)distance 

#### generate a simple linear dataset y=xb+e
##### store validation data from same dataset too

### create modeldict
- 'Ndiff_type': refers to the mathematical form of Ndiff
  - 'product'-Ndiff1 multiplied by Ndiff2
  - 'recursive'-Ndiff1's bw is Ndiff 2
- max_bw_Ndiff: is the depth of Ndiffs applied in estimating the bandwidth.
- 'normalize_Ndiffwtsum':
  - 'across' means sum across own level of kernelized-Ndiffs and divide by that sum (CDF approach)
  - 'own_n' means for (n+k)diff where n+K=max_bw_Ndiff
- Ndiff_bw_kern:
  - rbfkern means use the radial basis function kernel
  - 'product' means use product kernel like as in liu and yang eq1. 
- 'regression_model':
  - 'NW' means use nadaraya-watson kernel regression
  - 'NW-rbf' means use NW but calculate p(y,x) using rbf kernel rather than product kernel
  - 'full_logit' means local logit with all variables entering linearly
  - 'rbf_logit' means local logit with 1 parameters: scaled l2 norm centered on zero 
 (globally or by i?). Is this a new idea?
- outer_x_bw_form: if x is separated into blocks of rvars that are combined within a block using rbf kernel, 
  - 'one_for_all' - use same hx bw for all blocks
  - 'one_per_block' - each block or rvars gets an hx
- ykern_grid
  - if int, then create int evnely spaced values from -3 to 3 (standard normal middle ~99%)
  - 'no' means use original data points (self is masked when predicting self)
- xkern_grid
  - much like ykern_grid, but used less frequently since x's are typically pre-specified
- product_kern_norm: when multiplying kernels across random variables, this parameter determines if each random variable has its kernels normalized before the product or not
  - 'self' means each random variable has its kernels divided by the sum of kernels across the nout axis (the number of possibile values y is averaged over, whose probabilities sum to 1)
  - 'own_n' means same as self, but divided by count of non-masked items in second to last, nout lenght dimension
  - 'none' means no normalization prior to taking products across rvar kernels
- hyper_param_form_dict is a nested dictionary
  - Ndiff_exponent is the exponent wrapped around typically a sum of kernels for each Ndiff level 
after level 1 (i.e., (i-j) or centered, obvious/conventional level.)
  - x_bandscale is the parameter specific to each variable (each x in X) used for prediction (y)
  - Ndiff_depth_bw is used as the kernel's bandwidth (h) at each level of Ndiff including at level 1 
  - outer_x_bw vanilla bandwidth for the rbf or product kernel
  - outer_y_bw vanilla bandwidth for y

In [1]:
import os
import kernelcompare as kc
#from importlib import reload
networkdir='o:/public/dpatton/kernel'
mydir=os.getcwd()
#mydir2=os.path.join(mydir,"old models without datagen")
test=kc.KernelCompare(directory=mydir)

handlername:mykern-None.log


In [2]:
n=32
ykerngrid_form_variations=('modeldict:ykerngrid_form',[('exp',5),('even',4)])
#lossfn_variations=('modeldict:loss_function',['batch_crossval'])
lossfn_variations=('modeldict:loss_function',['mse'])
Ndiff_type_variations=('modeldict:Ndiff_type',['recursive','product'])
max_bw_Ndiff_variations=('modeldict:max_bw_Ndiff',[2])
NWnorm_variations=('modeldict:NWnorm',['across'])
Ndiff_start_variations=('modeldict:Ndiff_start',[1])
ykern_grid_variations=('modeldict:ykern_grid',[n+1])
regression_model_variations=('modeldict:regression_model',['NW-rbf'])
#product_kern_norm_variations=('modeldict:product_kern_norm',['self','own_n'])#include None too?
#normalize_Ndiffwtsum_variations=('modeldict:normalize_Ndiffwtsum',['own_n','across'])
optdict_variation_list=[NWnorm_variations,
                        ykerngrid_form_variations, 
                        lossfn_variations,
                        regression_model_variations,
                        Ndiff_type_variations,
                        ykern_grid_variations,
                        max_bw_Ndiff_variations,Ndiff_start_variations
                       ]  #,product_kern_norm_variations,normalize_Ndiffwtsum_variations]



In [3]:
#the default datagen_dict as of 11/25/2019
#datagen_dict={'batch_n':32,'batchcount':10, 'param_count':param_count,'seed':1, 'ftype':'linear', 'evar':1, 'source':'monte'}


batch_n_variations=('batch_n',[n])
batchcount_variations=('batchcount',[7])
#ftype_variations=('ftype',['linear','quadratic'])
ftype_variations=('ftype',['quadratic'])
param_count_variations=('param_count',[4])
#need to add block capability to datagen so we can repeat optimization with new training data.

datagen_variation_list=[batch_n_variations,batchcount_variations,ftype_variations,param_count_variations]



In [4]:
testrun=test.prep_model_list(optdict_variation_list=optdict_variation_list,datagen_variation_list=datagen_variation_list,verbose=1)
'''print(f'len(testrun){len(testrun)}')
print(f'testrun[0]{testrun[0:2]}')
print('--------------')
print(f'testrun[-1]{testrun[-4:]}')'''

dict_combo_list has 1 variations to run
len(datagen_dict_list):1
dict_combo_list has 4 variations to run


"print(f'len(testrun){len(testrun)}')\nprint(f'testrun[0]{testrun[0:2]}')\nprint('--------------')\nprint(f'testrun[-1]{testrun[-4:]}')"

In [5]:
import traceback
#from random import shuffle
#shuffle(testrun)
#a_rundict=testrun[100]#this produced the Ndiff_exponent error for recursive Ndiff
for idx in range(1):#len(testrun)):
    print(f'~~~~~~~run number:{idx}`~~~~~~~')
    a_rundict=testrun[idx]
    print(f'a_rundict{a_rundict}')
    optimizedict=a_rundict['optimizedict']
    datagen_dict=a_rundict['datagen_dict']

    try:
        test.do_monte_opt(optimizedict,datagen_dict,force_start_params=0)
        test.open_condense_resave('model_save',verbose=0)
        test.merge_and_condense_saved_models(merge_directory=None,save_directory=None,condense=None,verbose=None)
    except:
        print('traceback for run',idx)
        print(traceback.format_exc())


~~~~~~~run number:0`~~~~~~~
a_rundict{'optimizedict': {'opt_settings_dict': {'method': 'Nelder-Mead', 'options': {'xatol': 0.5, 'fatol': 1, 'adaptive': True}, 'help_start': 1, 'partial_match': 1}, 'hyper_param_dict': {'Ndiff_exponent': array([1.e-05, 1.e-05]), 'x_bandscale': array([1., 1., 1., 1.]), 'outer_x_bw': array([0.3]), 'outer_y_bw': array([0.3]), 'Ndiff_depth_bw': array([0.5]), 'y_bandscale': array([1.])}, 'modeldict': {'loss_function': 'mse', 'Ndiff_type': 'recursive', 'param_count': 4, 'Ndiff_start': 1, 'max_bw_Ndiff': 2, 'normalize_Ndiffwtsum': 'own_n', 'xkern_grid': 'no', 'ykern_grid': 33, 'outer_kern': 'gaussian', 'Ndiff_bw_kern': 'rbfkern', 'outer_x_bw_form': 'one_for_all', 'regression_model': 'NW-rbf', 'product_kern_norm': 'self', 'hyper_param_form_dict': {'Ndiff_exponent': 'free', 'x_bandscale': 'non-neg', 'Ndiff_depth_bw': 'non-neg', 'outer_x_bw': 'non-neg', 'outer_y_bw': 'non-neg', 'y_bandscale': 'fixed'}, 'NWnorm': 'across', 'ykerngrid_form': ('exp', 5)}}, 'datagen_d